# 缓存

缓存：将一个某个views的返回值保存至内存或者memcache中，5分钟内再有人来访问时，则不再去执行view中的操作，而是直接从内存或者Redis中之前缓存的内容拿到，并返回。

Django中提供了6种缓存方式：

* 开发调试
* 内存
* 文件
* 数据库
* Memcache缓存（python-memcached模块）
* Memcache缓存（pylibmc模块）

Django默认不支持redis做缓存，需要第三方模块django-redis.

## 开发调试


这里的开发调试指的是用户按照正常写缓存的方式去写代码，该加装饰器加装饰器，该做什么缓存操作做什么缓存操作，程序不报错，但是实际上也并没有真正缓存什么，真实情况还是从数据库捞数据，只不过这种方式可以用于开发调试，等代码写好之后缓存条件(如memcached安装好)具备之后进行backend的切换。
```
# 此为开始调试用，实际内部不做任何操作
    # 配置：
        CACHES = {
            'default': {
                'BACKEND': 'django.core.cache.backends.dummy.DummyCache',     # 引擎
                'TIMEOUT': 300,                                               # 缓存超时时间（默认300，None表示永不过期，0表示立即过期）
                'OPTIONS':{
                    'MAX_ENTRIES': 300,                                       # 最大缓存个数（默认300）
                    'CULL_FREQUENCY': 3,                                      # 缓存到达最大个数之后，剔除缓存个数的比例，即：1/CULL_FREQUENCY（默认3）
                },
                'KEY_PREFIX': '',                                             # 缓存key的前缀（默认空）
                'VERSION': 1,                                                 # 缓存key的版本（默认1）
                'KEY_FUNCTION' 函数名                                          # 生成key的函数（默认函数会生成为：【前缀:版本:key】）
            }
        }


    # 自定义key
    def default_key_func(key, key_prefix, version):
        """
        Default function to generate keys.

        Constructs the key used by all other methods. By default it prepends
        the `key_prefix'. KEY_FUNCTION can be used to specify an alternate
        function with custom key making behavior.
        """
        return '%s:%s:%s' % (key_prefix, version, key)

    def get_key_func(key_func):
        """
        Function to decide which key function to use.

        Defaults to ``default_key_func``.
        """
        if key_func is not None:
            if callable(key_func):
                return key_func
            else:
                return import_string(key_func)
        return default_key_func
```

## 内存

```
# 此缓存将内容保存至内存的变量中
    # 配置：
        CACHES = {
            'default': {
                'BACKEND': 'django.core.cache.backends.locmem.LocMemCache',
                'LOCATION': 'unique-snowflake',
            }
        }

    # 注：其他配置同开发调试版本
```

## 文件

```
# 此缓存将内容保存至文件
    # 配置：

        CACHES = {
            'default': {
                'BACKEND': 'django.core.cache.backends.filebased.FileBasedCache',
                'LOCATION': '/var/tmp/django_cache',
            }
        }
    # 注：其他配置同开发调试版本
```

## 数据库

```
# 此缓存将内容保存至数据库

    # 配置：
        CACHES = {
            'default': {
                'BACKEND': 'django.core.cache.backends.db.DatabaseCache',
                'LOCATION': 'my_cache_table', # 数据库表
            }
        }

    # 注：执行创建表命令 python manage.py createcachetable
```

## Memcache缓存(python-memcached模块)

```
# 此缓存使用python-memcached模块连接memcache

    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
            'LOCATION': '127.0.0.1:11211',
        }
    }

    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
            'LOCATION': 'unix:/tmp/memcached.sock',
        }
    }   

    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.MemcachedCache',
            'LOCATION': [
                '172.19.26.240:11211',
                '172.19.26.242:11211',
            ]
        }
    }
```

## Memcache缓存(pylibmc模块)

```
# 此缓存使用pylibmc模块连接memcache
    
    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.PyLibMCCache',
            'LOCATION': '127.0.0.1:11211',
        }
    }

    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.PyLibMCCache',
            'LOCATION': '/tmp/memcached.sock',
        }
    }   

    CACHES = {
        'default': {
            'BACKEND': 'django.core.cache.backends.memcached.PyLibMCCache',
            'LOCATION': [
                '172.19.26.240:11211',
                '172.19.26.242:11211',
            ]
        }
    }
```

# 应用

## 全站使用

全部的view使用缓存，加中间件即可。对于缓存有2个操作，一个是从缓存里取值，一个是往缓存里放值。其中从缓存里取值的中间件应该放在最后，因为需要走csrf认证通过之后才应该可以去拿缓存的数据。往缓存里放值是在最前面，因为view返回的response是从后往前传递的，用户应该拿到的是最后一个response, 这样的response可能会加一些东西，如响应头等。

```
MIDDLEWARE = [
        'django.middleware.cache.UpdateCacheMiddleware',
        # 其他中间件...
        'django.middleware.cache.FetchFromCacheMiddleware',
    ]

CACHE_MIDDLEWARE_ALIAS = ""
CACHE_MIDDLEWARE_SECONDS = ""
CACHE_MIDDLEWARE_KEY_PREFIX = ""
```

## 单独视图缓存

    from django.views.decorators.cache import cache_page, never_cache

```
方式一：
    from django.views.decorators.cache import cache_page

    @cache_page(60 * 15)
    def my_view(request):
        ...

方式二：
    from django.views.decorators.cache import cache_page

    urlpatterns = [
        url(r'^foo/([0-9]{1,2})/$', cache_page(60 * 15)(my_view)),
    ]
    
```




# 低层次缓存API 

rom django.core.cache import cache 

基本的接口是 set(key, value, timeout_seconds) 和 get(key) 

timeout_seconds 参数是可选的, 并且默认为前面讲过的 CACHE_BACKEND 设置中的 timeout 参数. 如果对象在缓存中不存在, 或者缓存后端是不可达的, cache.get;() 返回 None。

要想一次获取多个缓存值，可以使用 cache.get_many()。如果可能的话，对于给定的缓存后端，get_many()将只访问缓存一次，而不是对每个缓存键值都进行一次访问。 get_many() 所返回的字典包括了你所请求的存在于缓存中且未超时的所有键值。

可以用 cache.delete() 显式地删除关键字。这是在缓存中清除特定对象的简单途径，cache.delete() 没有返回值, 不管给定的缓存关键字对应的值存在与否。

缺省情况下，Django 的缓存系统使用所请求的路径（比如： “/stories/2005/jun/23/bank_robbed/” ）来创建其缓存键。这意味着对该 URL 的每个请求都将使用同一个已缓存版本，而不考虑 cookies 或语言偏好之类的 user-agent; 差别。然而，如果该页面基于请求头标的区别（例如 cookies、语言或者 user-agent)产生不同内容，你就不得不使用 Vary 头标来通知缓存机制：该页面的输出取决与这些东西。

```
from django.views.decorators.vary import vary_on_headers 
@vary_on_headers('User-Agent', 'Cookie') 
def my_view(request): 
    ...
```
传入vary_on_headers头标是大小写不敏感的；User-Agent 与 user-agent 完全相同。

# 序列化

```
import json 
from datetime import date 
from datetime import datetime 
   
class JsonCustomEncoder(json.JSONEncoder): 
    
    def default(self, field): 
     
        if isinstance(field, datetime): 
            return o.strftime('%Y-%m-%d %H:%M:%S') 
        elif isinstance(field, date): 
            return o.strftime('%Y-%m-%d') 
        else: 
            return json.JSONEncoder.default(self, field) 
   
   
# ds = json.dumps(d, cls=JsonCustomEncoder) 
```